# Spark Consumer

TODO: 

-find how to make output to 2 places : save json to hdfs and table to hive

-how come sometimes there are 2 tweets in one output, like:
<pre>
+----+--------------------+  
| key|               value|  
+----+--------------------+  
|null|"@SenWarren Need ...|  
|null|"RT @TrinityResis...|  
+----+--------------------+  
</pre>
-how to consume from a kafka topic everything there is up until now (cursor)
-how to cope with shit like  <pre>\\ud83c\\uddf5\\ud83c\\uddf8\\u2764\\</pre> instead of arabic chars


In [ ]:
from __future__ import print_function

import sys
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


In [ ]:
topic = 'TweeterArchive'

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell' 

In [ ]:
import pyarrow as pa
fs = pa.hdfs.connect(
    host='localhost', 
    port=8020, 
    user='hdfs', 
    kerb_ticket=None, 
    driver='libhdfs', 
    extra_conf=None)

In [ ]:
type(fs)

In [ ]:
#USE WITH CAUTION

#DELETE and recreate the data and checkpoint directories in hdfs 

hdfs_archive_path = '/tmp/project/archive'

if fs.exists(hdfs_archive_path):
    fs.delete(hdfs_archive_path,recursive=True)
else:
    print("no such dir: " + hdfs_archive_path)
#fs.mkdir(hdfs_archive_path)

    
    
hdfs_archive_checkpoint_path = hdfs_archive_path + '/sparkcheckpoint'
# if fs.exists(hdfs_archive_checkpoint_path):
#     fs.delete(hdfs_archive_checkpoint_path,recursive=True)
    
# else:
#     print("no such dir: " + hdfs_archive_checkpoint_path)
fs.mkdir(hdfs_archive_checkpoint_path)    
print("directory created: " + fs.ls(hdfs_archive_path)[0])
fs.chmod(hdfs_archive_checkpoint_path, 0o777)   
fs.chmod(hdfs_archive_path, 0o777)    
    
    

In [ ]:
spark = SparkSession\
    .builder\
    .appName("StructuredTwitterJsonArchive")\
    .getOrCreate()

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
# Json to HDFS sink with partitioning

targetJsonHDFS = df\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .partitionBy("key")\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
targetJsonHDFS.awaitTermination()

In [ ]:
targetJsonHDFS.stop()

## Sandbox 
below are cells that I played with to test portions of code


In [ ]:
# How to see the output in juppyter:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

df.show()

In [ ]:
# json to HDFS sink with trigger

targetJsonHDFS = df\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
targetJsonHDFS.awaitTermination()

In [ ]:
targetJsonHDFS.stop()

In [ ]:
# parquet to HDFS sink example with trigger

targetParquetHDFS = df\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020/tmp/project")\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020/tmp/sparkcheckpoint/")\
    .start()
targetParquetHDFS.awaitTermination()

In [ ]:
targetParquetHDFS.stop()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
#output to console
# query = df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()
# query.awaitTermination()

In [ ]:
# words = df.select(
#    explode(
#        split(df.value, " ")
#    ).alias("word")
# )

# Generate running word count
#wordCounts = words.groupBy("word").count()

In [ ]:
# query = wordCounts \
#     .writeStream \
#     .outputMode("complete") \
#     .format("console") \
#     .start()
# query.awaitTermination()